# Lesson 2 - Assignment

In this assignment, you will implement a Decision Tree algorithm from scratch and compare the results to existing sklearn algorithm. 

In [1]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.legend_handler import HandlerLine2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

# make this notebook's output stable across runs
np.random.seed(0)

Question 1.1: Implement the functions to calculate Gini Index.

In [2]:
# Calculate the Gini index for a split dataset
# Group here is a portion of the split dataset for each attribute X
# Format of group is [[1,1], [0,0]] where the first 1 in 1,1 is the value of X and the second 1 is the corresponding output class y
# Classes are the unique classes in the groups
def gini_index(groups, classes):
    # TODO count all samples at split point
    n_samples = sum([len(group) for group in groups])
    # Reference: https://stackoverflow.com/questions/3780403/python-sum-string-lengths
    
    # TODO sum weighted Gini index for each group
    gini = 0
    for group in groups:
#         print("group:", group)
        size = len(group)
        # TODO avoid divide by zero
        if size == 0:
            continue
            
        score = 0
        # TODO score the group based on the score for each class
        for class_val in classes:
#             print("class val:", class_val)
            # Get the output y/last value for each row within each group and count instances of the current class value
            p = [g[-1] for g in group].count(class_val)/size
            score += (p*p)
        # TODO weight the group score by its relative size
        # Using the formula gini index = (1 - sum(p^2)) * (group size / sample size)
        gini += (1-score) * (size/n_samples)
    return gini

In [3]:
gini_index([[[1, 1], [1, 1]], [[1, 1], [1, 0]]], [0, 1])

0.25

Question 2.1: Write a method that splits the dataset

In [4]:
# Split a dataset based on an attribute and an attribute value
# Attribute is at index, value is the threshold value for lesser than, greater than split into right and left lists
def test_split(index, value, dataset):
    left, right = list(), list()
    """
    TODO: This function loops over each row and checks if the row belongs to the right or left list.
    """ 
    for row in dataset:
        #to do
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

Question 2.2: Write a method that loops over the dataset, determine the groups of rows that belong to the right or left split, and calculates the gini_index

In [5]:
def get_split(dataset):
    "TODO Select the best split point for a dataset"
    # Get last unique values of last column
    class_values = np.unique([row[-1] for row in dataset])
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                # Set the below variables to minimum values
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

Question 2.3: Repeat question 2.2 using entropy

In [6]:
# Calculate the entropy for a split dataset
# Group here is a portion of the split dataset for each attribute X
# Format of group is [[1,1], [0,0]] where the first 1 in 1,1 is the value of X and the second 1 is the corresponding output class y
# Classes are the unique classes in the groups
def entropy(groups, classes):
    # TODO count all samples at split point
    n_samples = sum([len(group) for group in groups])
    
    # TODO sum weighted Gini index for each group
    entropy = 0
    for group in groups:
#         print("group:", group)
        score = 0
        size = len(group)
        # TODO avoid divide by zero
        if size == 0:
            continue
        # TODO score the group based on the score for each class
        for class_val in classes:
#             print("class val:", class_val)
            # Get the output y/last value for each row within each group and count instances of the current class value
            p = [g[-1] for g in group].count(class_val)/size
#             print("p:", p)
            # Using the formula entropy = -(sum(p * log2 p))
            if p != 0: # avoid log of 0
                score = (p * np.log2(p))
#             print("score:", score)
        entropy -= score
    return entropy

In [7]:
entropy([[[1, 1], [1, 1]], [[1, 1], [1, 0]]], [0, 1])

0.5

In [8]:
def get_split_entropy(dataset):
    "TODO Select the best split point for a dataset"
    class_values = np.unique([row[-1] for row in dataset])
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            entropy_ = entropy(groups, class_values)
            if entropy_ < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], entropy_, groups # changed gini here to entropy
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

Question 3.1: Write a method that takes in a group of rows and determines the class they belongs to. It should return the most common output value for a list of rows.

In [9]:
# This function is per group, not multiple groups
# Make a final prediction for the group by finding mode of class outcome
def to_terminal(group):
    "TODO determing the most common output within each group"
    outcomes = [g[-1] for g in group]
    return max(set(outcomes), key=outcomes.count)
    # Reference: https://stackoverflow.com/questions/10797819/finding-the-mode-of-a-list

In [10]:
to_terminal([[1, 1], [1, 1]])

1

Question 3.2: Write a method that recursively split the data.
The method takes in a node, max_depth, min_size, and depth. Initially, the method would be called by passing the root node and depth of 1. Here are the steps to help you implement:

- First, we create two groups for the data split and delete any existing groups from the node. As rows are used, they are no longer needed.
- Second, check if rows should be in left or right group, and if so we create a terminal node using the records we have.
- Third, check if maximum depth is reached and if so we create a terminal node.
- Fourth, process left child, creating a terminal node if the group of rows is too small, otherwise creating and adding the left node in a depth first fashion until the bottom of the tree is reached on this branch.
- Fifth, process the right side in a similar manner as left side, as we rise back up the constructed tree to the root.

In [11]:
# Create child splits for a node or make terminal
# Node is the current node, depth is current depth of node, max depth is max depth of tree, min_size is number of row samples to be processed per node
# Node is a dictionary that has groups, index and value
# Index definition corrected after discord chat: index is the feature X's index that the node is splitting on
# Value is the splitting value between left and right
def split(node, max_depth, min_size, depth):
    print("----------")
    print("node index: ", node['index'], "node value: ", node['value'])
    left, right = node['groups']
    print("node left length: ", len(left))
    print("node right length: ", len(right))
    del(node['groups'])
    # TODO check for a no split
    if not left or not right:
        print("Not left or right")
        node['left'] = node['right'] = to_terminal(right+left)
        return
    # TODO check for max depth
    if depth >= max_depth:
        print("Max depth")
        node['left'] = to_terminal(left)
        node['right'] = to_terminal(right)
        return
    # TODO process left child
    if len(left) <= min_size:
        print("End processing left sub tree")
        node['left'] = to_terminal(left)
    else:
        print("Recurse on left sub tree")
        node['left'] = get_split(left)
#         node['left'] = get_split_entropy(left)
        split(node['left'], max_depth, min_size, depth+1)
    # TODO process right child
    if len(right) <= min_size:
        print("End processing right sub tree")
        node['right'] = to_terminal(right)
    else:
        print("Recurse on right sub tree")
        node['right'] = get_split(right)
#         node['right'] = get_split_entropy(right)
        split(node['right'], max_depth, min_size, depth+1)

Question 3.3: Write a method that builds the tree. The method creates an initial split to determine root node, and then calls the split method.

In [12]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    "TODO get the first split, and then split starting fromt the root"
    root = get_split(train)
#     root = get_split_entropy(train)
    split(root, max_depth, min_size, 1)
    return root

Question 3.4: Write a method that takes in a node and rows of data, and predicts the class associated with each row.

In [13]:
# Make a prediction with a decision tree
def predict(node, row):
    #TODO check if a row belongs to a node and recursively traverse the tree if the row doesn't.
    if row[node['index']] < node['value']: # go left if row attribute X value is lesser than node value else go right
        if isinstance(node['left'], dict): # if non terminal node
            return predict(node['left'], row) # recurse to left subtree
        else:
            return node['left'] # hit terminal node
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row) # recurse to right subtree
        else:
            return node['right'] # hit terminal node

Question 4: Train a decision tree using the banknote_authentication data

In [14]:
from random import seed
from random import randrange
from csv import reader
from sklearn.metrics import accuracy_score

# Load a CSV file
def load_csv(filename):
    file = open(filename, "rt")
    lines = reader(file)
    dataset = list(lines)
    return dataset
 
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
filename = 'data_banknote_authentication.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
    
train = dataset[1:np.int64(len(dataset)*2/3)]
test = dataset[np.int64(len(dataset)*2/3)+1:len(dataset)]

In [15]:
#TODO Build a tree and evalute accuracy
max_depth = 7
min_size = 3
tree = build_tree(train, max_depth, min_size)
predictions = list()
for row in test:
    prediction = predict(tree, row)
    predictions.append(prediction)
               
print('Accuracy: %s' % accuracy_score([row[-1] for row in test], predictions))

----------
node index:  0 node value:  -0.27802
node left length:  207
node right length:  706
Recurse on left sub tree
----------
node index:  1 node value:  7.886
node left length:  142
node right length:  65
Recurse on left sub tree
----------
node index:  2 node value:  6.2204
node left length:  89
node right length:  53
Recurse on left sub tree
----------
node index:  0 node value:  -0.53966
node left length:  78
node right length:  11
Recurse on left sub tree
----------
node index:  0 node value:  -1.3971
node left length:  53
node right length:  25
Recurse on left sub tree
----------
node index:  0 node value:  -2.2804
node left length:  29
node right length:  24
Recurse on left sub tree
----------
node index:  0 node value:  -5.9034
node left length:  1
node right length:  28
Max depth
Recurse on right sub tree
----------
node index:  0 node value:  -2.2804
node left length:  0
node right length:  24
Not left or right
Recurse on right sub tree
----------
node index:  0 node val

In [16]:
from sklearn.metrics import f1_score
print('F1 score: %s' % f1_score([row[-1] for row in test], predictions))

F1 score: 0.9833147942157954


In [17]:
def print_tree(node, depth):
    if isinstance(node, dict):
        print('%s[%d <-- %.3f]' % (depth*' ', node['index'], node['value']))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print(depth*' ', "class", node)

In [18]:
print_tree(tree, 0)

[0 <-- -0.278]
 [1 <-- 7.886]
  [2 <-- 6.220]
   [0 <-- -0.540]
    [0 <-- -1.397]
     [0 <-- -2.280]
      [0 <-- -5.903]
        class 1.0
        class 1.0
      [0 <-- -2.280]
        class 1.0
        class 1.0
     [0 <-- -1.397]
       class 1.0
       class 1.0
    [1 <-- 4.116]
     [0 <-- -0.365]
      [0 <-- -0.475]
        class 1.0
        class 1.0
      [0 <-- -0.365]
        class 1.0
        class 1.0
      class 0.0
   [1 <-- -4.606]
    [0 <-- -1.668]
     [0 <-- -3.848]
      [0 <-- -4.855]
        class 1.0
        class 1.0
      [0 <-- -3.848]
        class 1.0
        class 1.0
     [0 <-- -1.668]
       class 1.0
       class 1.0
    [0 <-- -1.616]
     [0 <-- -1.695]
       class 0.0
       class 0.0
     [0 <-- -1.616]
       class 0.0
       class 0.0
  [0 <-- -4.286]
   [0 <-- -5.490]
     class 1.0
     class 1.0
   [0 <-- -1.577]
    [0 <-- -2.742]
      class 0.0
     [0 <-- -2.742]
       class 0.0
       class 0.0
    [0 <-- -1.577]
      class 0.0
  

[Bonus] Question 5: Train and evaluate an SKLEARN decision tree model, and compare the results to your model 

In [19]:
# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit([x[:-1] for x in train], [y[-1] for y in train])
clf_pred = clf.predict([x[:-1] for x in test])
acc_clf = accuracy_score([row[-1] for row in test], clf_pred)
print("Scikit Learn accuracy:", acc_clf)
print("Tree depth:", clf.get_depth())
print('CLF F1 score: %s' % f1_score([row[-1] for row in test], clf_pred))

Scikit Learn accuracy: 0.9452954048140044
Tree depth: 7
CLF F1 score: 0.9718785151856019


In [20]:
# Scaling data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train = scaler.fit_transform([x[:-1] for x in train])
x_test = scaler.transform([x[:-1] for x in test])

In [21]:
clf_sc = DecisionTreeClassifier(random_state=0)
clf_sc.fit(x_train, [y[-1] for y in train])
clf_pred_sc = clf.predict(x_test)
acc_clf_sc = accuracy_score([row[-1] for row in test], clf_pred_sc)
print("Scikit Learn scaled accuracy:", acc_clf_sc)
print("Tree depth:", clf_sc.get_depth())
print('CLF F1 score scaled: %s' % f1_score([row[-1] for row in test], clf_pred_sc))

Scikit Learn scaled accuracy: 0.9890590809628009
Tree depth: 7
CLF F1 score scaled: 0.9944994499449945


In [22]:
# Converting scaled features data set to complete data set including target y as input to decision tree
x_train = x_train.tolist()
for i, row in enumerate(train):
    x_train[i].append(train[i][-1])
    
x_test = x_test.tolist()
for i, row in enumerate(test):
    x_test[i].append(test[i][-1])
    
# x_test[0]

In [23]:
max_depth = 7
min_size = 3
tree = build_tree(x_train, max_depth, min_size)
predictions_sc = list()
for row in x_test:
    prediction = predict(tree, row)
    predictions_sc.append(prediction)
               
print('Accuracy scaled: %s' % accuracy_score([row[-1] for row in x_test], predictions_sc))
print('F1 score scaled: %s' % f1_score([row[-1] for row in test], predictions_sc))

----------
node index:  0 node value:  0.4672831728317283
node left length:  207
node right length:  706
Recurse on left sub tree
----------
node index:  1 node value:  0.80848031153708
node left length:  142
node right length:  65
Recurse on left sub tree
----------
node index:  2 node value:  0.5005427556291824
node left length:  89
node right length:  53
Recurse on left sub tree
----------
node index:  0 node value:  0.44765997659976603
node left length:  78
node right length:  11
Recurse on left sub tree
----------
node index:  0 node value:  0.38335133351333517
node left length:  53
node right length:  25
Recurse on left sub tree
----------
node index:  0 node value:  0.3171031710317103
node left length:  29
node right length:  24
Recurse on left sub tree
----------
node index:  0 node value:  0.0453754537545375
node left length:  1
node right length:  28
Max depth
Recurse on right sub tree
----------
node index:  0 node value:  0.3171031710317103
node left length:  0
node right l

In [24]:
print_tree(tree, 0)

[0 <-- 0.467]
 [1 <-- 0.808]
  [2 <-- 0.501]
   [0 <-- 0.448]
    [0 <-- 0.383]
     [0 <-- 0.317]
      [0 <-- 0.045]
        class 1.0
        class 1.0
      [0 <-- 0.317]
        class 1.0
        class 1.0
     [0 <-- 0.383]
       class 1.0
       class 1.0
    [1 <-- 0.666]
     [0 <-- 0.461]
      [0 <-- 0.453]
        class 1.0
        class 1.0
      [0 <-- 0.461]
        class 1.0
        class 1.0
      class 0.0
   [1 <-- 0.336]
    [0 <-- 0.363]
     [0 <-- 0.200]
      [0 <-- 0.124]
        class 1.0
        class 1.0
      [0 <-- 0.200]
        class 1.0
        class 1.0
     [0 <-- 0.363]
       class 1.0
       class 1.0
    [0 <-- 0.367]
     [0 <-- 0.361]
       class 0.0
       class 0.0
     [0 <-- 0.367]
       class 0.0
       class 0.0
  [0 <-- 0.167]
   [0 <-- 0.076]
     class 1.0
     class 1.0
   [0 <-- 0.370]
    [0 <-- 0.282]
      class 0.0
     [0 <-- 0.282]
       class 0.0
       class 0.0
    [0 <-- 0.370]
      class 0.0
      class 0.0
 [2 <-- 0.0

Question 6: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: What was your incoming experience with this model, if any? what steps you took, what obstacles you encountered. how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?) This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.

Observations:
Entropy gives lesser accuracy than Gini.
Sample size 1-3 gives the highest accuracy after which accuracy decreases.
Tree depth greater than 7 (tested until 13) gives highest accuracy.
Implemented decision tree performs better at 96.71% in terms of accuracy when compared to Scikit Learn's decision tree classifier at 94.53%. Scikit Learn's classifier improves to 98.9% when data is scaled. Scaling data does not seem to affect accuracy of implemented decision tree.

Incoming experience: No incoming experience apart from previous assignments and a data structure knowledge of trees and decision trees

Steps taken: This week's lesson was about regression and classification using decision trees Implemented binary classification on the data set to get a deeper understanding of the concepts. Started with implementing bottom up i.e. implemented load data, predict, build tree etc 

Obstacles: The challenge here was to figure out why the functions had the inputs that they did, what their data types are and how to test each function to make sure data passes seamlessly from one function to another. Some errors thrown took a while to figure out

Link to real world: Helped me understand how decision trees are built, what each node contains in terms of its information and the flow to how a decision/prediction is made

Steps missing (with just this week's learning): Data needed to be scaled